In [13]:
import os
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms
from PIL import Image


In [14]:
class UTKFaceDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        self.root_dir = root_dir
        self.image_paths = [os.path.join(root_dir, fname) for fname in os.listdir(root_dir) if fname.endswith('.jpg')]
        self.transform = transform

    def __len__(self):
        return len(self.image_paths)

    def __getitem__(self, idx):
        image_path = self.image_paths[idx]
        image = Image.open(image_path).convert('RGB')
        age = int(os.path.basename(image_path).split('_')[0])
        if self.transform:
            image = self.transform(image)
        return image, torch.tensor(age, dtype=torch.float32)


In [15]:
class AgeCNN(nn.Module):
    def __init__(self):
        super(AgeCNN, self).__init__()
        self.conv1 = nn.Conv2d(3, 32, 3)
        self.conv2 = nn.Conv2d(32, 64, 3)
        self.conv3 = nn.Conv2d(64, 128, 3)
        self.pool = nn.MaxPool2d(2, 2)
        self.fc1 = nn.Linear(128 * 10 * 10, 64)  # Will adjust if needed
        self.fc2 = nn.Linear(64, 1)

    def forward(self, x):
        x = F.relu(self.conv1(x))
        x = self.pool(x)
        x = F.relu(self.conv2(x))
        x = self.pool(x)
        x = F.relu(self.conv3(x))
        x = self.pool(x)
        print("Shape before FC:", x.shape)
        x = x.view(x.size(0), -1)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x


In [16]:
transform = transforms.Compose([
    transforms.Resize((100, 100)),
    transforms.ToTensor()
])

data_dir = r'C:\Users\Ketaki\Downloads\part2'  # Update if needed
dataset = UTKFaceDataset(data_dir, transform=transform)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)


In [17]:
model = AgeCNN()
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

for epoch in range(5):
    for images, labels in dataloader:
        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs.view(-1), labels)
        loss.backward()
        optimizer.step()
    print(f"Epoch {epoch+1}, Loss: {loss.item():.4f}")


Shape before FC: torch.Size([32, 128, 10, 10])
Shape before FC: torch.Size([32, 128, 10, 10])
Shape before FC: torch.Size([32, 128, 10, 10])
Shape before FC: torch.Size([32, 128, 10, 10])
Shape before FC: torch.Size([32, 128, 10, 10])
Shape before FC: torch.Size([32, 128, 10, 10])
Shape before FC: torch.Size([32, 128, 10, 10])
Shape before FC: torch.Size([32, 128, 10, 10])
Shape before FC: torch.Size([32, 128, 10, 10])
Shape before FC: torch.Size([32, 128, 10, 10])
Shape before FC: torch.Size([32, 128, 10, 10])
Shape before FC: torch.Size([32, 128, 10, 10])
Shape before FC: torch.Size([32, 128, 10, 10])
Shape before FC: torch.Size([32, 128, 10, 10])
Shape before FC: torch.Size([32, 128, 10, 10])
Shape before FC: torch.Size([32, 128, 10, 10])
Shape before FC: torch.Size([32, 128, 10, 10])
Shape before FC: torch.Size([32, 128, 10, 10])
Shape before FC: torch.Size([32, 128, 10, 10])
Shape before FC: torch.Size([32, 128, 10, 10])
Shape before FC: torch.Size([32, 128, 10, 10])
Shape before 

KeyboardInterrupt: 

In [25]:
import matplotlib.pyplot as plt
import numpy as np

# Load one image from the dataset
sample_img, _ = dataset[0]  # just the image, we don’t need the label here
sample_img = sample_img.unsqueeze(0)  # Add batch dimension (1, C, H, W)


In [27]:
# Dictionary to store activations
activations = {}

def get_activation(name):
    def hook(model, input, output):
        activations[name] = output.detach()
    return hook

# Register hook on conv1 (you can do conv2/conv3 later too)
model.conv1.register_forward_hook(get_activation('conv1'))
model.conv2.register_forward_hook(get_activation('conv2'))
model.conv3.register_forward_hook(get_activation('conv3'))




In [35]:
torch.save(model.state_dict(), 'age_cnn_weights.pt')

In [ ]:
def plot_activations(layer_name, num_filters=8, image_id=None):
    act = activations[layer_name].squeeze(0)
    
    fig, axes = plt.subplots(1, num_filters, figsize=(20, 5))
    for i in range(num_filters):
        axes[i].imshow(act[i].cpu(), cmap='viridis')
        axes[i].axis('off')
    plt.suptitle(f'Activation Maps from {layer_name}', fontsize=16)
    
    if image_id is not None:
        filename = f"activation_image{image_id}_{layer_name}.png"
        plt.savefig(filename)
        plt.close(fig)  # This is important to save and release memory
        print(f"Saved {filename}")
    else:
        plt.show()


In [ ]:
import os
print("Current working directory:", os.getcwd())


In [ ]:
import os

save_dir = "activation_maps"
os.makedirs(save_dir, exist_ok=True)

def plot_activations(layer_name, num_filters=8, image_id=None):
    act = activations[layer_name].squeeze(0)
    
    fig, axes = plt.subplots(1, num_filters, figsize=(20, 5))
    for i in range(num_filters):
        axes[i].imshow(act[i].cpu(), cmap='viridis')
        axes[i].axis('off')
    plt.suptitle(f'Activation Maps from {layer_name}', fontsize=16)
    
    if image_id is not None:
        filename = os.path.join(save_dir, f"activation_image{image_id}_{layer_name}.png")
        plt.savefig(filename)
        plt.close(fig)
        print(f"Saved {filename}")
    else:
        plt.show()


In [ ]:
for i in range(5):
    img, _ = dataset[i]
    img = img.unsqueeze(0)

    model.eval()
    with torch.no_grad():
        _ = model(img)

    print(f"Processing Image {i+1}")
    plot_activations('conv1', image_id=i+1)
    plot_activations('conv2', image_id=i+1)
    plot_activations('conv3', image_id=i+1)


In [ ]:
# Get the activation maps from conv1
act = activations['conv1'].squeeze() 

# Plot first 8 activation maps
fig, axes = plt.subplots(1, 8, figsize=(20, 5))
for idx in range(8):
    axes[idx].imshow(act[idx].cpu(), cmap='viridis')
    axes[idx].axis('off')
plt.suptitle("Activation Maps from conv1", fontsize=16)
plt.show()
